In [ ]:
%load_ext autoreload
%autoreload 2

import yaml

import torch
import pandas as pd
from IPython.core.display import HTML
from IPython.display import display
from torch.nn.functional import normalize

from oml.utils.visualisation import RetrievalVisualizer
from oml.lightning.entrypoints.validate import main
from oml.functional.metrics import calc_retrieval_metrics
from oml.lightning.callbacks.metric import MetricValCallback

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline


In [ ]:
cfg = """

    dataset_root:  /nydl/data/DeepFashion_InShop/
    dataframe_name: df_fixed_val.csv
    logs_root: logs
    im_size: 224
    bs_val: 128
    num_workers: 20

    model:
      name: vit
      args:
        arch: vits16
        normalise_features: False
        use_multi_scale: False
        weights: vits16_inshop
        strict_load: True

    metric_args:
      cmc_top_k: [1]
      map_top_k: [5]

    gpus:
      - 0
  
"""
        
trainer, ret_dict = main(yaml.load(cfg, Loader=yaml.Loader));

clb_metric = [x for x in trainer.callbacks if isinstance(x, MetricValCallback)][0]
vis = RetrievalVisualizer.from_embeddings_metric(clb_metric.metric)


In [ ]:
n_query = clb_metric.metric.distance_matrix.shape[0]

for i in range(min(10, n_query)):
    vis.visualise(query_idx=i, top_k=1, skip_no_errors=False)
